In [1]:
import geopandas as gpd
from shapely import geometry
import warnings
warnings.filterwarnings('ignore')

In [2]:
gdf = gpd.read_file("RI_Precinct.json")

In [3]:
gdf.head()

,OBJECTID,NAME,Shape__Are,Shape__Len,geometry
0,1,0102,0.000348,0.153037,"MULTIPOLYGON (((-71.34122 41.73016, -71.34123 ..."
1,2,0101,0.000426,0.107569,"POLYGON ((-71.32270 41.77270, -71.32270 41.772..."
2,3,0104,0.000509,0.338228,"MULTIPOLYGON (((-71.30425 41.74515, -71.30425 ..."
3,4,0105,0.000520,0.297727,"MULTIPOLYGON (((-71.29308 41.71355, -71.29307 ..."
4,5,0103,0.000584,0.158078,"POLYGON ((-71.31979 41.73751, -71.31897 41.737..."


In [4]:
gdf.drop(["OBJECTID", "Shape__Are", "Shape__Len"], axis=1, inplace=True)

In [5]:
gdf.rename({"NAME":"Precinct #"}, axis=1, inplace=True)

In [6]:
gdf.sort_values(by=["Precinct #"], ignore_index=True, inplace=True)

In [7]:
gdf.head(1)

,Precinct #,geometry
0,0101,"POLYGON ((-71.32270 41.77270, -71.32270 41.772..."


# fix precinct multipolygon error

In [10]:
gdf["new_geo"] = ""

In [11]:
gdf.head(1)

,Precinct #,geometry,new_geo,index1,index2
0,0101,"POLYGON ((-71.32270 41.77270, -71.32270 41.772...",,,


In [168]:
# shape with area < 0.00001 will be dropped
area_threshold = 0.0000001
for index, row in gdf.iterrows():
    cur_geometry = row["geometry"]
    polygon_list = []
    if cur_geometry.type == "MultiPolygon":
        for shape in cur_geometry:
            if shape.area >= area_threshold:
                polygon_list.append(shape)
        if len(polygon_list) == 1:
            gdf.at[index, "new_geo"] = polygon_list[0]
        else:
            new_multipoly = geometry.multipolygon.MultiPolygon(polygon_list)
            gdf.at[index, "new_geo"] = new_multipoly
    else:
        gdf.at[index, "new_geo"] = cur_geometry

In [169]:
gdf.set_geometry("new_geo", inplace=True, drop=True)

In [170]:
gdf.to_file("RI_Precinct_Cleaned.json", driver="GeoJSON")